Create .env file with the following vars

```
OPENAI_API_TYPE = azure
OPENAI_API_VERSION = 2023-03-15-preview
OPENAI_API_BASE = ' ... ' # Replace with the URL of an Azure OpenAI gpt-3.5-turbo deployment
OPENAI_API_KEY = ' ... ' # Replace with the corresponding API key

```

In [ ]:

# Install required packages
import sys

#!conda install --yes --prefix {sys.prefix} numpy
!conda install --yes --prefix {sys.prefix} -c conda-forge langchain
!conda install --yes --prefix {sys.prefix} python-dotenv
!conda install --yes --prefix {sys.prefix} -c conda-forge openai 
!conda install --yes --prefix {sys.prefix} -c conda-forge tiktoken
!conda install --yes --prefix {sys.prefix} -c conda-forge pypdf
!conda install --yes --prefix {sys.prefix} -c conda-forge chromadb

In [ ]:
import os
from dotenv import load_dotenv
from langchain.document_loaders import PyPDFLoader # for loading the pdf
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

load_dotenv()

loader = PyPDFLoader('power-platform-2023-release-wave-1-plan.pdf')

# combine all text
pages = loader.load_and_split()

embeddings = OpenAIEmbeddings(deployment="text-embedding-ada-002", chunk_size=1)

vectordb = Chroma.from_documents(pages, embedding=embeddings,
                                 persist_directory="./vectordb")

#vectordb.persist()




In [7]:
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import AzureChatOpenAI


llm = AzureChatOpenAI(deployment_name="gpt-35-turbo", temperature=0.9)

pdf_qa = ConversationalRetrievalChain.from_llm(llm, vectordb.as_retriever())


In [9]:
query = "When can we expect Microsoft Dataverse to go in Public preview?"

result = pdf_qa({"question": query, "chat_history": ""})

print("Answer:")
print(result["answer"])


Answer:
Based on the information provided in the context, Microsoft Dataverse is already in Public preview.


In [10]:
query = "List all of the features going into public preview in May 2023?"

result = pdf_qa({"question": query, "chat_history": ""})

print("Answer:")
print(result["answer"])

Answer:
There is only one feature going into public preview in May 2023 according to the context given. The feature is "Test Power Apps in CICD pipelines".


In [ ]:
query = "List all of the features going into public preview in Sep 2023?"

result = pdf_qa({"question": query, "chat_history": ""})

print("Answer:")
print(result["answer"])